In [1]:
libraries<-c("reshape","reshape2","ggthemes","tidyverse","data.table","stringr")
suppressPackageStartupMessages(lapply(libraries, require, character.only = TRUE))

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

In [2]:
# crear carpetas para datos output y setear directorio de trabajo para Lbraziliensis 

path_lbra<-"/media/eduardo/D1/ncRNAs_Leishmania_spp/RNAseq/Lbraziliensis/analysis/06_WGCNA/unsigned/tables/"

setwd(path_lbra)

In [3]:
#cargar datos
module_membership <- read.delim("geneModuleMembership.csv", sep ='\t', header=TRUE)
module_significance <- read.delim("PvaluegeneModuleMembership.csv", sep='\t', header= TRUE) 

In [4]:
#generar columna geneid con los nombres de los genes derivado del rownames de cada archivo 
module_membership$geneid<-rownames(module_membership)
module_significance$geneid<-rownames(module_significance)

In [5]:
mm_melted<-reshape2::melt(module_membership,id.vars = "geneid")

In [6]:
colnames(mm_melted)<-c("geneid","module","mm")

In [7]:
mmsig_melted<-reshape2::melt(module_significance,id.vars = "geneid")

In [8]:
colnames(mmsig_melted)<-c("geneid","module","p_mm")

In [9]:
mmsig_melted$module <- gsub("p.", "", mmsig_melted$module, fixed = TRUE)


In [10]:
head(mm_melted)
head(mmsig_melted)

,geneid,module,mm
,<chr>,<fct>,<dbl>
1,LbrM.10.1540.1,MMblue,0.7351566
2,ncRNA00001,MMblue,0.8294405
3,LbrM.10.1550.1,MMblue,0.9191505
4,LbrM.10.1560.1,MMblue,0.8873347
5,LbrM.10.1570.1,MMblue,0.7962717
6,LbrM.10.1580.1,MMblue,0.7319801


,geneid,module,p_mm
,<chr>,<chr>,<dbl>
1,LbrM.10.1540.1,MMblue,3.284422e-07
2,ncRNA00001,MMblue,4.075737e-10
3,LbrM.10.1550.1,MMblue,2.575113e-15
4,LbrM.10.1560.1,MMblue,5.643965e-13
5,LbrM.10.1570.1,MMblue,6.359547e-09
6,LbrM.10.1580.1,MMblue,3.914348e-07


In [11]:
#merge tablas
merged_mm <- merge(mm_melted, mmsig_melted, by = c("geneid","module"))

In [12]:
head(merged_mm)

,geneid,module,mm,p_mm
,<chr>,<fct>,<dbl>,<dbl>
1,LbrM.01.0010,MMblack,-0.75861170,8.289862e-08
2,LbrM.01.0010,MMblue,0.97555427,5.879095e-24
3,LbrM.01.0010,MMbrown,-0.88700060,5.919559e-13
4,LbrM.01.0010,MMgreenyellow,0.09378923,5.863884e-01
5,LbrM.01.0010,MMgrey,-0.12144884,4.804402e-01
6,LbrM.01.0010,MMlightcyan,-0.55398132,4.560779e-04


In [13]:
# filtro por (module membership) mm y pvalue mm
filtered_mm<-subset(merged_mm, mm >= 0.85 & p_mm < 0.00001)

In [14]:
head(filtered_mm)

,geneid,module,mm,p_mm
,<chr>,<fct>,<dbl>,<dbl>
2,LbrM.01.0010,MMblue,0.9755543,5.879095e-24
13,LbrM.01.0020,MMbrown,0.9113428,1.160894e-14
22,LbrM.01.0030,MMblue,0.9542145,2.143286e-19
32,LbrM.01.0040,MMblue,0.8982812,1.083191e-13
72,LbrM.01.0080,MMblue,0.9871573,1.136116e-28
84,LbrM.01.0090,MMgreenyellow,0.9306824,2.059591e-16


In [15]:
#ingresar archivos gs de wgcna por tratamiento
#GS Amastigote
gs_ama<-read.delim("geneTraitSignificance_amastigote.csv",sep = '\t', header=TRUE)
gsp_ama<-read.delim("GeneSignificancePvalue_amastigote.csv",sep = '\t', header=TRUE)

In [16]:
#generar columna geneid con los nombres de los genes derivado del rownames de cada archivo 
gs_ama$geneid<-rownames(gs_ama)
gsp_ama$geneid<-rownames(gsp_ama)

In [17]:
#tabla merged con datos gs y gsp
merged_gs_ama <- merge(gs_ama,gsp_ama, by = "geneid")

In [18]:
head(merged_gs_ama)

,geneid,GS.Amastigote,p.GS.Amastigote
,<chr>,<dbl>,<dbl>
1,LbrM.01.0010,0.8702906,5.401656e-12
2,LbrM.01.0020,-0.4225061,1.025710e-02
3,LbrM.01.0030,0.9757341,5.192608e-24
4,LbrM.01.0040,0.7869909,1.255301e-08
5,LbrM.01.0050,0.7779002,2.367366e-08
6,LbrM.01.0060,0.7863298,1.315919e-08


In [19]:
#filtros estadísticos
filtered_ama<-subset(merged_gs_ama, GS.Amastigote >= 0.7 & p.GS.Amastigote < 0.01)

In [20]:
#tabla merged entre la tabla anterior filtrada y datos de membership. 6columns
ama_genes <- merge(filtered_ama,filtered_mm, by = "geneid")

In [21]:
ama_genes

geneid,GS.Amastigote,p.GS.Amastigote,module,mm,p_mm
<chr>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>
LbrM.01.0010,0.8702906,5.401656e-12,MMblue,0.9755543,5.879095e-24
LbrM.01.0030,0.9757341,5.192608e-24,MMblue,0.9542145,2.143286e-19
LbrM.01.0040,0.7869909,1.255301e-08,MMblue,0.8982812,1.083191e-13
LbrM.01.0080,0.9192135,2.542495e-15,MMblue,0.9871573,1.136116e-28
LbrM.01.0110,0.9321607,1.444215e-16,MMblue,0.8901305,3.764553e-13
LbrM.01.0120,0.8844931,8.426758e-13,MMblue,0.8515175,4.621702e-11
LbrM.01.0150,0.8819737,1.191891e-12,MMblue,0.8716156,4.584949e-12
LbrM.01.0170,0.9215511,1.571910e-15,MMblue,0.8888010,4.570159e-13
LbrM.01.0200,0.9433696,7.313685e-18,MMblue,0.9078005,2.197484e-14


In [22]:
#guardar
write.table(ama_genes, "amastigote_genes_filtrados.tsv",sep = '\t',row.names = FALSE,quote = FALSE)

In [23]:
#GS metacyclic
gs_meta <- read.delim("geneTraitSignificance_metacyclic.csv", sep = '\t', header=TRUE)
gsp_meta <- read.delim("GeneSignificancePvalue_metacyclic.csv", sep = '\t', header=TRUE)
gs_meta$geneid<-rownames(gs_meta)
gsp_meta$geneid<-rownames(gsp_meta)
merged_gs_meta <- merge(gs_meta,gsp_meta, by = "geneid")
filtered_meta<-subset(merged_gs_meta, GS.Metacyclic >= 0.7 & p.GS.Metacyclic < 0.01)
meta_genes <- merge(filtered_meta,filtered_mm, by = "geneid")
write.table(meta_genes, "metacyclic_genes_filtrados.tsv",sep = '\t',row.names = FALSE,quote = FALSE)
#6columnas
length(meta_genes$geneid)

[1] 320

In [24]:
#GS procyclic
gs_pro <- read.delim("geneTraitSignificance_procyclic.csv", sep = '\t', header=TRUE)
gsp_pro <- read.delim("GeneSignificancePvalue_procyclic.csv", sep = '\t', header=TRUE)
gs_pro$geneid<-rownames(gs_pro)
gsp_pro$geneid<-rownames(gsp_pro)
merged_gs_pro <- merge(gs_pro,gsp_pro, by = "geneid")
filtered_pro<-subset(merged_gs_pro, GS.Procyclic >= 0.7 & p.GS.Procyclic < 0.01)
pro_genes <- merge(filtered_pro,filtered_mm, by = "geneid")
write.table(pro_genes, "Procyclic_genes_filtrados.tsv",sep = '\t',row.names = FALSE,quote = FALSE)
#6columnas
length(pro_genes$geneid)

[1] 1467